In [31]:
from tqdm.auto import tqdm
import time
import subprocess
import random

In [32]:
wa = False
from typing import List

class Test():
    def __init__(self):
        self.generate_test()
        
    def generate_test(self): 
        with open("in/test.in", 'w') as test:
            self.n = random.randrange(32, 1000)
            
            test.write(f"{self.n}\n")
            
            nums = list(range(1, self.n + 1))
            random.shuffle(nums)
            
            for num in nums:
                test.write(f"{num} ")
            test.write("\n")
        
    def check_solutions(self, a_out: str, b_out: str):
        global wa
        # a: brut, b: better_solution
        with open(a_out, 'r') as a, open(b_out, 'r') as b:
            self.brut_data = [int(i) for i in a.read().strip().split()]
            self.chad_data = [int(i) for i in b.read().strip().split()]
                
        self.better_check()
    
    def better_check(self):
        global wa
        
        if self.brut_data != self.chad_data:
            print("WA")
            wa = True
            return

In [33]:
# convention a: brut, b: better_solutoin
class Referee():
    def __init__(self, program_a: str, program_b: str, max_time: int):
        # program a = sth.cpp (give full name with .cpp)
        
        self.program_a = program_a
        self.program_b = program_b
        self.max_time = max_time
        self.times: List[List[float]] = []
        
        # compile programs 
        self.exe_a = self.compile(program_a)
        self.exe_b = self.compile(program_b)
        
    def compile(self, program: str) -> str:
        exe_name = program.split('.')[0]
        
        compile_command = [
            'g++',
            f'src/{program}',
            '-o', f'bin/{exe_name}',
            '-std=gnu++17',
            '-O2',
        ]
        subprocess.run(compile_command, check=True)
        return exe_name
    
    def do_tests(self): 
        self.times = []
        for _ in tqdm(range(100_000)): 
            # create test 
            self.test = Test()
            
            # test program_a
            start_time_a = time.time()
            with open("in/test.in", "r") as test_input, open(f"out/{self.exe_a}.out", "w") as output_a:
                try:
                    subprocess.run(f"./bin/{self.exe_a}", stdin=test_input, stdout=output_a, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_a} exceeded time limit")
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_a} ended with an error")
                    exit(2)
            end_time_a = time.time()
            
            # test program_b
            start_time_b = time.time()
            with open("in/test.in", "r") as test_input, open(f"out/{self.exe_b}.out", "w") as output_b:
                try:
                    subprocess.run(f"./bin/{self.exe_b}", stdin=test_input, stdout=output_b, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_b} exceeded time limit") 
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_b} ended with an error")
                    exit(2)
            end_time_b = time.time()
            
            
            # check solution
            global wa
            wa = False
            self.test.check_solutions(f"out/{self.exe_a}.out", f"out/{self.exe_b}.out")
            if wa:
               break 
            
            # solutions are ok
            self.times.append([end_time_a - start_time_a, end_time_b - start_time_b])
        
        else:
            print("everything ok 🚀") 

In [34]:
referee = Referee('brut_solution.cpp', 'gigachad_solution.cpp', 2)
referee.do_tests()

100%|██████████| 100000/100000 [05:00<00:00, 332.85it/s]

everything ok 🚀


In [35]:
import pandas as pd
df = pd.DataFrame(referee.times)
df.columns = ['brut', 'chad']
df = df.sort_values(by='chad', ascending=False)
df

,brut,chad
91255,0.001308,0.007859
71262,0.001341,0.007791
74591,0.001456,0.007727
8029,0.001344,0.007719
61281,0.001390,0.007662
...,...,...
74206,0.001333,0.001242
53969,0.001351,0.001240
13223,0.001344,0.001240
33961,0.001302,0.001240


In [36]:
df.mean()
df.sort_values(by='brut', ascending=False)

,brut,chad
71257,0.007714,0.001377
61276,0.007677,0.001389
67945,0.007661,0.001386
24638,0.007594,0.001389
94589,0.007577,0.001402
...,...,...
16091,0.001256,0.001378
54306,0.001256,0.001294
78664,0.001254,0.001364
53696,0.001254,0.001278
